## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Tokenize and numericalize your samples
def vectorize_en(text, vocab, sequence_length):
    tokens = tokenizer(text)
    tokens = [vocab[token] for token in tokens] + [vocab["<eos>"]]
    token_ids = tokens[:sequence_length] + [vocab["<pad>"]] * (sequence_length - len(tokens))
    return token_ids

def vectorize_vn(text, vocab, sequence_length):
    tokens = tokenizer(text)
    tokens = [vocab["<sos>"]] + [vocab[token] for token in tokens] + [vocab["<eos>"]]
    token_ids = tokens[:sequence_length] + [vocab["<pad>"]] * (sequence_length - len(tokens))
    return token_ids

In [3]:
corpus_en = [
    "good morning",
    "ai books"    
]
data_size_en = len(corpus_en)

# max vocabulary size and sequence length
vocab_size_en = 7
sequence_length_en = 3

In [4]:
# Create vocabulary
vocab_en = build_vocab_from_iterator(yield_tokens(corpus_en),
                                     max_tokens=vocab_size_en,
                                     specials=["<unk>", "<pad>", "<eos>"])
vocab_en.set_default_index(vocab_en["<unk>"])
vocab_en.get_stoi()

{'morning': 6,
 'good': 5,
 'books': 4,
 'ai': 3,
 '<eos>': 2,
 '<pad>': 1,
 '<unk>': 0}

In [5]:
# Vectorize the samples
corpus_ids_en = []
for sentence in corpus_en:
    corpus_ids_en.append(vectorize_en(sentence, vocab_en, sequence_length_en))

# print
en_data = torch.tensor(corpus_ids_en, dtype=torch.long)
print(en_data)    

tensor([[5, 6, 2],
        [3, 4, 2]])


In [6]:
corpus_vn = [
    "chào buổi sáng",
    "sách ai"    
]
data_size_vn = len(corpus_vn)

# max vocabulary size and sequence length
vocab_size_vn = 9
sequence_length_vn = 4

In [7]:
# Create vocabulary
vocab_vn = build_vocab_from_iterator(yield_tokens(corpus_vn),
                                  max_tokens=vocab_size_vn,
                                  specials=["<unk>", "<pad>", "<sos>", "<eos>"])
vocab_vn.set_default_index(vocab_vn["<unk>"])
vocab_vn.get_stoi()

{'sách': 7,
 'sáng': 8,
 'chào': 6,
 'buổi': 5,
 '<sos>': 2,
 'ai': 4,
 '<eos>': 3,
 '<pad>': 1,
 '<unk>': 0}

In [8]:
# Vectorize the samples
corpus_ids_vn = []
for sentence in corpus_vn:
    corpus_ids_vn.append(vectorize_vn(sentence, vocab_vn, sequence_length_vn+1))

# print
print(corpus_ids_vn)

[[2, 6, 5, 8, 3], [2, 7, 4, 3, 1]]


In [9]:
input_vn_data = []
label_vn_data = []

for vector in corpus_ids_vn:
    input_vn_data.append(vector[:-1])
    label_vn_data.append(vector[1:])  

# convert to tensors
input_vn_data = torch.tensor(input_vn_data, dtype=torch.long)
label_vn_data = torch.tensor(label_vn_data, dtype=torch.long)

# print
print(input_vn_data)
print(label_vn_data)

tensor([[2, 6, 5, 8],
        [2, 7, 4, 3]])
tensor([[6, 5, 8, 3],
        [7, 4, 3, 1]])


## Model

In [10]:
class Encoder(nn.Module):
    def __init__(self, vocab_size_en, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size_en, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

    # src: [batch_size, seq_length]
    def forward(self, src): 
        embedded = self.embedding(src)  # [batch_size, seq_length, embedding_dim]        
        _, hidden = self.rnn(embedded)  # [1, batch_size, hidden_dim]        
        return hidden

In [11]:
embedding_dim, hidden_dim = 6, 6
encoder = Encoder(vocab_size_en, embedding_dim, hidden_dim)

hidden_sample = encoder(en_data)
print(hidden_sample.shape)

torch.Size([1, 2, 6])


In [12]:
class Decoder(nn.Module):
    def __init__(self, vocab_size_vn, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size_vn, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, vocab_size_vn)

    # input: [batch_size, seq_length]
    # hidden: [1, batch_size, hidden_dim]
    def forward(self, input, hidden):
        embedded = self.embedding(input)          # [batch_size, seq_length, embedding_dim]        
        output, _ = self.rnn(embedded, hidden)    # [batch_size, seq_length, hidden_dim]
        prediction = self.fc_out(output)          # [batch_size, vocab_size_vn]
        
        return prediction.permute(0, 2, 1)

In [13]:
decoder = Decoder(vocab_size_vn, embedding_dim, hidden_dim)
outputs = decoder(input_vn_data, hidden_sample)
print(outputs.shape)

torch.Size([2, 9, 4])


In [14]:
class Seq2Seq_Model(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, sequence_en, sequence_vn):        
        hidden = self.encoder(sequence_en)
        outputs = self.decoder(sequence_vn, hidden)
            
        return outputs

In [15]:
model = Seq2Seq_Model(encoder, decoder)
print(model)

Seq2Seq_Model(
  (encoder): Encoder(
    (embedding): Embedding(7, 6)
    (rnn): RNN(6, 6, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(9, 6)
    (rnn): RNN(6, 6, batch_first=True)
    (fc_out): Linear(in_features=6, out_features=9, bias=True)
  )
)


In [16]:
outputs = model(en_data, input_vn_data)
print(outputs.shape)

torch.Size([2, 9, 4])


## Train

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [18]:
for _ in range(35):
    optimizer.zero_grad()
    outputs = model(en_data, input_vn_data)
    loss = criterion(outputs, label_vn_data)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.1774535179138184
1.8258389234542847
1.5507405996322632
1.3226356506347656
1.1275200843811035
0.9576416611671448
0.8097606897354126
0.6848278641700745
0.5825912952423096
0.4985010623931885
0.42496055364608765
0.35829031467437744
0.30022522807121277
0.2523270547389984
0.2137191891670227
0.1825522631406784
0.15717321634292603
0.13620106875896454
0.11843530833721161
0.10286650061607361
0.08932499587535858
0.07957738637924194
0.07216261327266693
0.0642516240477562
0.05687557905912399
0.05110391601920128
0.0466657355427742
0.04282408952713013
0.039204519242048264
0.0357925146818161
0.03275236487388611
0.03022100031375885
0.028152476996183395
0.026374174281954765
0.024742424488067627


In [19]:
outputs = model(en_data, input_vn_data)
print(torch.argmax(outputs, axis=1))

tensor([[6, 5, 8, 3],
        [7, 4, 3, 1]])


In [20]:
label_vn_data

tensor([[6, 5, 8, 3],
        [7, 4, 3, 1]])